# SIT744 Assignment 4 (T2 2025)


## P-Level Tasks (~60%)

Goal: Evaluate Pretrained Model’s Generative Baseline

1. Model & Domain Selection

    . Choose a small, widely accessible GPT‑style model (e.g., GPT‑2 small or DistilGPT‑2).

    . Pick a distinctive writing domain or author (e.g., Jane Austen’s novels, satirical news articles, scientific abstracts).

i will chose DistilGPT-2 due to faster, smaller model with fewer computational resources


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random
import numpy as np

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)


model_id = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

2. Data Preparation

    . Collect ~5–10 representative samples from your chosen domain.
    
    . Keep them short and manageable (e.g., one paragraph each).

My domain sample is scientific abstract that is related to machine learning in healthcare.


In [ ]:
domain_sample = [
    "Machine learning (ML) and its applications in healthcare have gained a lot of attention. When enhanced computational power is combined with big data, there is an opportunity to use ML algorithms to improve health care. Supervised learning is the type of ML that can be implemented to predict labeled data based on algorithms such as linear or logistic regression, support vector machine, decision tree, LASSO regression, K Nearest Neighbor, and Naive Bayes classifier. Unsupervised ML models can identify data patterns in datasets that do not contain information about the outcome. Such models can be used for fraud or anomaly detection. Examples of clinical applications of ML include the formulation of various clinical decision support systems. An important public health application of ML is the identification and prediction of populations at high risk for developing certain adverse health outcomes and the development of public health interventions targeted to these populations. Various concepts related to ML need to be integrated into the medical curriculum so that health professionals can effectively guide and interpret research in this area.",
    "Machine learning (ML) is a study of computer algorithms for automation through experience. ML is a subset of artificial intelligence (AI) that develops computer systems, which are able to perform tasks generally having need of human intelligence. While healthcare communication is important in order to tactfully translate and disseminate information to support and educate patients and public, ML is proven applicable in healthcare with the ability for complex dialogue management and conversational flexibility. In this topical review, we will highlight how the application of ML/AI in healthcare communication is able to benefit humans. This includes chatbots for the COVID-19 health education, cancer therapy, and medical imaging. ",
    "The increasing availability of electronic health data presents a major opportunity in healthcare for both discovery and practical applications to improve healthcare. However, for healthcare epidemiologists to best use these data, computational techniques that can handle large complex datasets are required. Machine learning (ML), the study of tools and methods for identifying patterns in data, can help. The appropriate application of ML to these data promises to transform patient risk stratification broadly in the field of medicine and especially in infectious diseases. This, in turn, could lead to targeted interventions that reduce the spread of healthcare-associated pathogens. In this review, we begin with an introduction to the basics of ML. We then move on to discuss how ML can transform healthcare epidemiology, providing examples of successful applications. Finally, we present special considerations for those healthcare epidemiologists who want to use and apply ML",
    "Recent advancements in Artificial Intelligence (AI) and Machine Learning (ML) technology have brought on substantial strides in predicting and identifying health emergencies, disease populations, and disease state and immune response, amongst a few. Although, skepticism remains regarding the practical application and interpretation of results from ML-based approaches in healthcare settings, the inclusion of these approaches is increasing at a rapid pace. Here we provide a brief overview of machine learning-based approaches and learning algorithms including supervised, unsupervised, and reinforcement learning along with examples. Second, we discuss the application of ML in several healthcare fields, including radiology, genetics, electronic health records, and neuroimaging. We also briefly discuss the risks and challenges of ML application to healthcare such as system privacy and ethical concerns and provide suggestions for future applications.",
    "The drive towards greater penetration of machine learning in healthcare is being accompanied by increased calls for machine learning and AI based systems to be regulated and held accountable in healthcare. Interpretable machine learning models can be instrumental in holding machine learning systems accountable. Healthcare offers unique challenges for machine learning where the demands for explainability, model fidelity and performance in general are much higher as compared to most other domains. In this paper we review the notion of interpretability within the context of healthcare, the various nuances associated with it, challenges related to interpretability which are unique to healthcare and the future of interpretability in healthcare."
]

3. Baseline Generation

    . Use a fixed prompt (e.g., “Continue this paragraph in the style of …”). You can use a prompt like this:
        GEN_PROMPT = (
            "Continue the following passage in the same style:\n\n"
            "The results indicate a statistically significant improvement over baseline models under limited compute budgets."
        )

    . Generate continuations using the pretrained model without any fine‑tuning.
    
    Discussion on the steps take for tokenization and giving prompt:
    1. Prompt Creation: i used a fixed prompt to guide the model to continue in the style of scientific abstracts, this ensure consistency in testing the generative ability of the pretrained model.
    2. Tokenization: The prompt was converted into token id using the tokenizer. Tokenizer convert text to numberic ID that enable the model to process. The return_tensors="pt" argument ensures the output is a PyTorch tensor, compatible with the model.
    
    3. Attention Mask & Pad Token Handling: GPT-2 model has no pad_token by default, so i set
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    The attention mask is automatically created to tells the model which tokens are real input and which are padding. This prevents unexpected behavior during generation

    
    4. Text generation: i use the generate() method with parameters controlling randomness and repetition.
    - input_ids: prompt converted into numbers (token IDs).
    - attention_mask: tells the model which IDs are meaningful.
    - max_new_tokens=80:  continuation token is limit to 80.
    - do_sample=True: instead of always picking the single most likely next word, the model samples from a probability distribution.
    - temperature=0.7: lowers randomness , text stays coherent.
    - top_k=50: trims the candidate pool to top 50 likely words.
    - top_p=0.95: nucleus sampling , only keep tokens whose combined probability more than 95%.
    - repetition_penalty=1.2: discourages repeating the same phrases
    - pad_token_id=tokenizer.eos_token_id: fixes GPT-2’s lack of pad token, avoids warnings.

    5. Decoding
    Finally token id was convert back to original readable text, provide a human-readable continuation of the prompt in the same style.



In [ ]:
GEN_PROMPT = (
    "Continue the following passage in the same style:\n\n"
    "The results indicate a statistically significant improvement over baseline models under limited compute budgets."
)

inputs = tokenizer(GEN_PROMPT, return_tensors="pt").to(device)

import time

start = time.time()
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=80,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)
end = time.time()

# Decode to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)
print(f"Inference time (P-task baseline): {end - start:.2f} seconds")

Continue the following passage in the same style:

The results indicate a statistically significant improvement over baseline models under limited compute budgets. This is not surprising given that we have seen similar increases since 1980, and it seems unlikely at this time to be sustained for long term gains as well. As always, I would like to see these findings confirmed by additional studies of different types of computational tasks such (e-mailing or writing) with respect more frequently than those presented earlier.
Inference time (P-task baseline): 1.24 seconds


4. Evaluation

    . Qualitatively assess style similarity. (write a short analysis on what you see on generated data, in comparison with actual data, in terms of grammar and similarity)

The generated text maintains a professional, scientific tone, consistent with the original prompt. Grammar and sentence structure are generally correct, and the model replicates the surface style of scientific writing by using phrases such as “statistically significant” and “baseline models.”

Compared with the actual healthcare ML samples, the continuation shows some stylistic similarity in its formal, academic phrasing. However, unlike the domain samples, it lacks domain-specific vocabulary and instead introduces irrelevant examples (etc; “e-mailing or writing”) that would not typically appear in real abstracts. This highlights that the pretrained model captures the general style of scientific text but not the substance of the healthcare ML domain, which is expected since it was not fine-tuned on domain sample.

Overall, the output represents a reasonable baseline for evaluation, demonstrating fluent academic style but limited domain grounding, which could be improved with fine-tuning.

    . Search for a evaluation metric and compute this simple metric for the generated data (e.g., perplexity on held‑out domain samples, or BLEU/ROUGE comparing to original domain variants).

In [ ]:
# Perplexity Function
def compute_perplexity(model, tokenizer, text):
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

# Perplexity of Generated Text
ppl_generated = compute_perplexity(model, tokenizer, generated_text)
print(f"Perplexity of generated text: {ppl_generated:.2f}")


# Perplexity of Domain Samples=
import numpy as np

domain_ppls = []
for i, sample in enumerate(domain_sample, 1):
    ppl = compute_perplexity(model, tokenizer, sample)
    domain_ppls.append(ppl)
    print(f"Sample {i} Perplexity: {ppl:.2f}")

avg_domain_ppl = np.mean(domain_ppls)
print(f"\nAverage Domain Perplexity: {avg_domain_ppl:.2f}")


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Perplexity of generated text: 35.29
Sample 1 Perplexity: 37.34
Sample 2 Perplexity: 61.41
Sample 3 Perplexity: 30.36
Sample 4 Perplexity: 35.40
Sample 5 Perplexity: 40.89

Average Domain Perplexity: 41.08


Perplexity measures how surprised the model is by a given text. Lower values indicate the text is more predictable to the model, while higher values indicate greater difficulty. The generated text has a perplexity of 35.29, which is reasonable for a small GPT-2 model on scientific text it has not been fine-tuned on.

Comparing with domain samples, the generated text’s perplexity is similar to Samples 1, 3, and 4 (30–40 range), suggesting a reasonable stylistic match. Sample 5 is moderately higher (40.89) but still close to the expected range. However, Sample 2 shows a much higher perplexity (61.41), indicating vocabulary or structure less familiar to DistilGPT-2, pointing to a domain mismatch.

In short, the results are quite decent, since perplexity on in-domain text for GPT-2 typically falls in the 20–40 range, and most of our samples fall near this level. To further reduce perplexity and make the outputs more domain-natural, the next step would be fine-tuning GPT-2 on the target domain.


## C-Level Tasks (~30%)

Goal: Adaptation via Fine-Tuning

1. Fine-tune the model (full-tuning) on your domain samples. Make choices (number of epochs, learning rate, context length, etc.) that fit within your computing environment.
2. Generate same prompt continuation as the pass task.
3. Re-evaluate the performance on sample generation. Compare style, fluency, and compare based on the evaluation metric you chose before.  
4. Track the resources and report. Record the trainable parameters, training time, inference time.





In [ ]:
#prepare dataset
from datasets import Dataset

# Convert domain samples into Hugging Face Dataset
dataset = Dataset.from_dict({"text": domain_sample})

In [ ]:
#tokenization
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Fix pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

tokenized_ds = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
# training setup
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,             # few epochs (small dataset)
    per_device_train_batch_size=2,  # small batch
    learning_rate=5e-5,             # conservative LR
    weight_decay=0.01,
    save_strategy="no",
    logging_steps=1,
    report_to="none",
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)


/tmp/ipython-input-2523752729.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#train model
import time

start = time.time()
trainer.train()
end = time.time()

print(f"Training time: {end - start:.2f} seconds")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss
1,3.642500
2,3.647300
3,4.158200
4,3.494100
5,3.174800
6,3.400000
7,3.029900
8,3.398800
9,3.276300


Training time: 1.55 seconds


In [ ]:
#generate with fine tune model
inputs = tokenizer(GEN_PROMPT, return_tensors="pt").to(model.device)

start = time.time()
outputs = model.generate(
    **inputs,
    max_new_tokens=80,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)
end = time.time()

print("Fine-tuned continuation:\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"\nInference time: {end - start:.2f} seconds")


Fine-tuned continuation:

Continue the following passage in the same style:

The results indicate a statistically significant improvement over baseline models under limited compute budgets. Our work will offer further evidence that we can improve our understanding of how to optimize for optimal performance across different datasets, such as general population and machine learning (GIS). We are now exploring data structures on functional languages using GIs with high-quality computational power or computing algorithms within C++ libraries which need more than adequate statistical analysis at best.[1] The findings demonstrate an important distinction between language processing

Inference time: 0.94 seconds


In [ ]:
# Reevaluate the performance on sample generation.
print("=== Perplexities After Fine-Tuning ===")
ppl_generated = compute_perplexity(model, tokenizer, GEN_PROMPT)
print(f"Generated text PPL: {ppl_generated:.2f}")

domain_ppls = []
for i, sample in enumerate(domain_sample, 1):
    ppl = compute_perplexity(model, tokenizer, sample)
    domain_ppls.append(ppl)
    print(f"Sample {i} PPL: {ppl:.2f}")

avg_domain_ppl = np.mean(domain_ppls)
print(f"\nAverage Domain Perplexity: {avg_domain_ppl:.2f}")


=== Perplexities After Fine-Tuning ===
Generated text PPL: 111.03
Sample 1 PPL: 24.88
Sample 2 PPL: 42.14
Sample 3 PPL: 17.30
Sample 4 PPL: 22.21
Sample 5 PPL: 24.59

Average Domain Perplexity: 26.22


5. Introduce one modification of your own design (for example, a different hyperparameter setting, data sampling approach or model variant) and summarise its effect (performing the fine-tuning, sample generation, reevaluation and tracking the computational cost same as steps 1 to 4).


In [ ]:
# Training setup with modified learning rate
training_args_mod = TrainingArguments(
    output_dir="./results_mod",
    overwrite_output_dir=True,
    num_train_epochs=3,             # keep same epochs
    per_device_train_batch_size=2,
    learning_rate=1e-4,             # MODIFIED learning rate
    weight_decay=0.01,
    save_strategy="no",
    logging_steps=1,
    report_to="none",
    seed=42
)

trainer_mod = Trainer(
    model=AutoModelForCausalLM.from_pretrained(model_id),  # reload fresh model
    args=training_args_mod,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Fine-tune with modified setup
import time
start = time.time()
trainer_mod.train()
end = time.time()
print(f"Modified training time: {end - start:.2f} seconds")

# Generate with modified model
inputs = tokenizer(GEN_PROMPT, return_tensors="pt").to(model.device)

start = time.time()
outputs = trainer_mod.model.generate(
    **inputs,
    max_new_tokens=80,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)
end = time.time()

print("\nModified fine-tuned continuation:\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"\nModified inference time: {end - start:.2f} seconds")

# Evaluate with perplexity
print("\n=== Perplexities After Modified Fine-Tuning ===")
ppl_generated = compute_perplexity(trainer_mod.model, tokenizer, GEN_PROMPT)
print(f"Generated text PPL: {ppl_generated:.2f}")

domain_ppls = []
for i, sample in enumerate(domain_sample, 1):
    ppl = compute_perplexity(trainer_mod.model, tokenizer, sample)
    domain_ppls.append(ppl)
    print(f"Sample {i} PPL: {ppl:.2f}")

avg_domain_ppl = np.mean(domain_ppls)
print(f"\nAverage Domain Perplexity: {avg_domain_ppl:.2f}")


/tmp/ipython-input-3289619774.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_mod = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss
1,3.642500
2,3.638800
3,4.146000
4,3.352600
5,2.876100
6,3.274100
7,2.656500
8,3.066200
9,2.979000


Modified training time: 3.38 seconds

Modified fine-tuned continuation:

Continue the following passage in the same style:

The results indicate a statistically significant improvement over baseline models under limited compute budgets. A large proportion of those who are not involved in this study do so because they have difficulty interpreting data, which may lead to errors and inaccuracies for general populations (eg., diabetes). However it is important that statistical analyses can be used as an exploratory tool where we discuss such issues with our patients before undertaking clinical trials or at-risk studies based on these findings; otherwise information about them will likely

Modified inference time: 1.16 seconds

=== Perplexities After Modified Fine-Tuning ===
Generated text PPL: 111.52
Sample 1 PPL: 19.01
Sample 2 PPL: 28.76
Sample 3 PPL: 11.36
Sample 4 PPL: 15.38
Sample 5 PPL: 15.84

Average Domain Perplexity: 18.07


6. Discuss the trade-off between computational cost and sample generation performance based on the mothods you performed in this task.

| Method                 | Trainable Params | Training Time (s) | Inference Time (s) | Generated PPL | Domain PPL (avg) | Qualitative Description       |
| ---------------------- | ---------------- | ----------------- | ------------------ | ------------- | ---------------- | ----------------------------- |
| **Baseline (P-task)**  | 0                | 0                 | 1.24             | 35.29     | 41.08        | Fluent, generic               |
| **Full FT (5e-5)**     | 82M            | 1.55             | 0.94              | 111.03        | 26.22            | Domain-ish (somewhat related to the domain, but not exactly or fully aligned), incoherent        |
| **Modified FT (1e-4)** | 82M            | 3.38              | 1.16              | 111.52        | 18.07    | Domain-aligned, more coherent |


1. fine-tuning procedure and rationale  
The five short healthcare on machine learning domain sample are fine tuned with DistilGPT-2 (82 paramater). Fine tuned was chosen to evaluate whether the model could adapt to the target domain and surpass the performance of the baseline. Training used
- epoch(3): kept low because the dataset is tiny (only 5 domainsample), more epochs would likely overfit.
- batch size (2): small to reduce memory use and ensure more gradient update
- learning rate (5e-5, then increased to 1e-4 for the modification): 5e-5 provide stable update, while 1e-4 allow the model to learn more, tested to accelerate adaption
- max sequence length (256): chosen to cover the longest paragraph .

The rationale was to test domain allignment changes under the same small data conditions and different learning rate.


2. regenerated prompt comparison
- Baseline (P-task): Fluent and grammatically correct, but generic and off-topic (etc; e-mailing or writing).
- Fine tuned (lr=5e-5): Introduced technical jargon and domain terms (machine learning, functional languages, C++ libraries) but with less coherence and healthcare focus. the output was more technical but less readable.
- Modified fine tuned (lr=1e-4): More coherent and domain alligned (healthcare focus; mentions diabetes, patients, statistical analyses, clinical trials), but still less fluent than baseline.

This shows a trade off, fine tuning improve domain specificity but at the cost of fluency, especially with very small dataset (only 5 domain samples).


3. computational resource comparison
- Baseline: no training needed, inference is 1.24 s.
- Fine-tuned (5e-5): training took 1.55s, inference 0.94s. Generated PPL worsened, but domain PPL improved.  
- Modified fine-tuned (1e-4):training took 3.38s, inference 1.16s. Domain PPL improved further, but generated PPL remained high.  

Fine tuning require more computation but produced lower domain perplexity. However, this came at the cost of higher generated perplexity and reduced fluency on unseen prompts. In other words, the model become more domain specialized but less generalizable, indicating overfitting when training on very few samples, hence D task will focus on PEFT to reduce compute cost and avoid overfitting.


## D-Level Tasks (~10%)

Goal: Parameter-Efficient Fine-Tuning (PEFT)


1. Apply a parameter-efficient adaptation method (for example LoRA or adapter modules) to your model. Discuss the justification behind choosing an specific form of PEFT.

LoRA introduces low-rank adapter matrices into transformer attention layers while freezing the majority of pretrained weights. This enables the model to preserve general fluency while adapting to a new domain and have lower risk of overfitting.  

2. Apply chosen method using your domain data. Generate same prompt continuation. Re-evaluate performance on sample generation. Compare style, fluency, and compare based on the evaluation metric you chose before.



In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Reload base model
model = AutoModelForCausalLM.from_pretrained(model_id)

# Define LoRA config
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                 # rank of LoRA matrices (trade-off between cost/performance)
    lora_alpha=32,
    bias="all"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

# Count trainable params
model.print_trainable_parameters()


trainable params: 198,912 || all params: 82,060,032 || trainable%: 0.2424


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
training_args_lora = TrainingArguments(
    output_dir="./results_lora",
    overwrite_output_dir=True,
    num_train_epochs=3,             # keep same epochs
    per_device_train_batch_size=2,
    learning_rate=5e-4,             # MODIFIED learning rate
    weight_decay=0.01,
    save_strategy="no",
    logging_steps=1,
    report_to="none",
    seed=42
)

trainer_lora = Trainer(
    model=model,
    args=training_args_lora,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

start = time.time()
trainer_lora.train()
end = time.time()
print(f"LoRA training time: {end - start:.2f} seconds")

# Generate continuation
inputs = tokenizer(GEN_PROMPT, return_tensors="pt").to(model.device)

start = time.time()
outputs = model.generate(
    **inputs,
    max_new_tokens=80,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)
end = time.time()

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nLoRA fine-tuned continuation:\n")
print(generated_text)
print(f"\nLoRA inference time: {end - start:.2f} seconds")

# Evaluate perplexity
print("\n=== Perplexities After LoRA Fine-Tuning ===")
print(f"Generated text PPL: {compute_perplexity(model, tokenizer, GEN_PROMPT):.2f}")

domain_ppls = []
for i, sample in enumerate(domain_sample, 1):
    ppl = compute_perplexity(model, tokenizer, sample)
    domain_ppls.append(ppl)
    print(f"Sample {i} PPL: {ppl:.2f}")

avg_domain_ppl = sum(domain_ppls) / len(domain_ppls)
print(f"\nAverage Domain PPL: {avg_domain_ppl:.2f}")

/tmp/ipython-input-3380451781.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss
1,3.642500
2,3.673200
3,4.169900
4,3.656100
5,3.521600
6,3.654300
7,3.447200
8,3.719200
9,3.603900


LoRA training time: 1.14 seconds

LoRA fine-tuned continuation:

Continue the following passage in the same style:

The results indicate a statistically significant improvement over baseline models under limited compute budgets. Our regression analysis is based on four-point performance and comparison of these two groups, with respect to overall model size (mean for both cases). All data are represented as follows; Table 1 shows an estimated mean trend from one year thereafter within each group by linear fit across all covariates throughout our test suite including three main effects that account only for statistical significance among those whose variance represents nonparametric changes

LoRA inference time: 0.92 seconds

=== Perplexities After LoRA Fine-Tuning ===
Generated text PPL: 110.06
Sample 1 PPL: 34.49
Sample 2 PPL: 60.07
Sample 3 PPL: 27.63
Sample 4 PPL: 32.04
Sample 5 PPL: 38.23

Average Domain PPL: 38.49


3. Compare resource use (trainable parameter count, training time) and performance against the full fine-tuning methods mentioned on the C task.  

| Method                 | Trainable Params | Training Time (s) | Inference Time (s) | Generated PPL | Domain PPL (avg) | Qualitative Description                |
| ---------------------- | ---------------- | ----------------- | ------------------ | ------------- | ---------------- | -------------------------------------- |
| **Baseline (P-task)**  | 0                | 0                 | 1.24               | 35.29         | 41.08            | Fluent, generic                        |
| **Full FT (5e-5)**     | 82M              | 1.55              | 0.94               | 111.03        | 26.22            | Domain-ish, incoherent, overfit        |
| **Modified FT (1e-4)** | 82M              | 3.38              | 1.16               | 111.52        | 18.47            | Domain-aligned but overfit |
| **LoRA (r=8, α=32 )**        | 0.16M (0.2%)    | 1.14              | 0.92               | 110.06        | 38.49            | Fluent, structured, moderate domain focus  |

LORA significantly reduce the number of trainable parameters. While fine tune updated all 82M parameter, LORA update only 0.16M(0.2%), making it far more lightweight. Training time was shorter for LORA as it consumed much less memory which would matter at scale, Inference was also slightly faster for LORA compared to the fine tuned one.

Performance-wise, modified FT achieved much lower domain perplexity (better adaption) but at the cost of very high generated perplexity (loss fluency). On the other hand, LORA achieve the best generated perplexity (110.06), indicate more structured generalization, but highest average domain perplexity, indicate weaker domain alignment but less overfitting.

4. Discuss the trade-off between computational cost and sample generation performance in comparison with previous methods in task C.
Key points to cover:

a) Parameter-Efficient Fine-Tuning procedure and rationale  

LORA inserted small low rank adapter matrices in attention layers, leaving most pretrained weight frozen. This mean the model preserves general fluency while advancing towards the target domain. It is specially suitable for small dataset and limited compute as its training is memory and time efficient and also avoid updating all model weight eventually decrease overfitting.

config used:
- small r=8 keep the update very lightweight
- α = 32 strengthen the adapter updates so they still have effect.
- bias="all", slightly increase capacity to help compensate for the tiny dataset.
- learning rate set to 5e-4 since LORA has far fewer trainable paramater, a higher learning rate helps the small adapter learn meaningful update quickly.

The rationale of LORA is designed to mitigate overfitting and reduce compute when adapting large scale real world large language models to small dataset.


b) regenerated prompt comparison  

- Fine tuned (lr=5e-5): Introduced technical jargon and domain terms (machine learning, functional languages, C++ libraries) but with less coherence and healthcare focus. the output was more technical but less readable.
- Modified fine tuned (lr=1e-4): More coherent and domain alligned (healthcare focus; mentions diabetes, patients, statistical analyses, clinical trials), but still less fluent than baseline (clear overfitting).
- LoRA: Produced a fluent, structured academic continuation with statistical terms (regression analysis, statistical trend). However, it was less healthcare-specific than the Modified fine tuned, reflecting weaker domain alignment but stronger preservation of general fluency.

Trade-off:

Fine Tuned = strong domain fit but incoherent, overfit text.

LoRA = weaker domain fit but better fluency and structure.

c) computational resource comparison

Fine tuned require updating all 82M parameter, resulting in higher memory use, longer training, and a greater risk of overfitting on small datasets. In contrast, LORA updates only 0.2% of parameters, making training much faster and lighter on compute, while inference also remains efficient. Although LORA achieved weaker domain adaptation (higher domain perplexity), it preserved fluency and structure better than fine-tuning. With only five domain samples, the fine-tuned models tend to memorize and overfit, whereas LORA maintained generalization despite adapting only partially.

With larger domain datasets, LoRA would be expected to reduce domain perplexity significantly while still retaining fluency, whereas fine-tuning would continue to risk overfitting and demand far higher computational cost (Wang & Li, 2023). That is why real world fine tuning almost always use PEFT such as LORA because it is cheaper, avoid overfitting, and still achieve strong domain adaption when sufficient data is available. Furthermore, hyperparameter can be tuned until performance benchmark are met.  

## References:

1. Wang, F., & Li, B. (2025). Leaner Training, Lower Leakage: Revisiting Memorization in LLM Fine-Tuning with LoRA. arXiv preprint arXiv:2506.20856.


